In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
from isaac.utils import get_cuda_device_if_available
device = get_cuda_device_if_available()
print(device)

cuda:0


In [3]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.utils import plot_confusion_matrix
from isaac.constants import BASIC_TRAINING_COLS, FORCE_CLASS_COLS, MASS_CLASS_COLS
from isaac.training import training_loop, evaluate_saved_model

In [4]:
BATCH_SIZE = 128
EPOCHS = 100
NORMALISE_DATA = True
STEP_SIZE = 3
SEQ_END = 1800

INPUT_DIM = len(BASIC_TRAINING_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

## Read dataset and preprocess it

In [5]:
train_trials = read_dataset("data/train_passive_trials.h5", n_trials=3500)
val_trials = read_dataset("data/val_passive_trials.h5", n_trials=800)

100%|██████████| 10/10 [00:00<00:00, 117.96it/s]


In [6]:
class MultiBranchModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout=0.5):
        super(MultiBranchModel, self).__init__()
        # RNN
        self.base_gru = nn.GRU(input_dim, hidden_dim, 2, batch_first=True, dropout=dropout)
        # Readout layer
        self.gru1 = nn.GRU(hidden_dim, hidden_dim, 2, batch_first=True, dropout=dropout)
        self.gru2 = nn.GRU(hidden_dim, hidden_dim, 2, batch_first=True, dropout=dropout)
        
        self.fc1 = nn.Linear(hidden_dim, output_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        out, _ = self.base_gru(x)
        
        out_1, _ = self.gru1(out)
        out_1 = self.fc1(out_1[:, -1, :]) 
        
        out_2, _ = self.gru2(out)
        out_2 = self.fc2(out_2[:, -1, :]) 
        return out_1, out_2

## Define model, loss and optimizer

In [7]:
def training_loop(model, optimizer, mass_error, force_error, train_loader, val_loader, num_epochs=200, print_stats_per_epoch=True,
                  seq_start=None, seq_end=None, step_size=None):    
    """Trains a model for <num_epochs> to minimize the <error> using the <optimizer>.
    Returns a list of epoch losses (averaged over batches) as well as validation accuracy"""
    
    best_model, best_mass_accuracy, best_force_accuracy = None, 0, 0
    epoch_losses = [[], []]
    epoch_accuracies = [[],[]]
    
    pbar = tqdm(range(num_epochs))
    
    for epoch in pbar:
        model.train()
        
        mass_total_loss = 0.
        force_total_loss = 0.

        for x, y in train_loader:

            x = Variable(x[:, seq_start:seq_end:step_size, :])
            y = Variable(y)
            
            mass_y = y[:, 0]
            force_y = y[:, 1]
            
            # Clear gradients
            optimizer.zero_grad()
            # Forward propagation
            mass_y_hat, force_y_hat = model(x)
            # Calculate softmax and cross entropy loss
            mass_loss = mass_error(mass_y_hat, mass_y)
            force_loss = force_error(force_y_hat, force_y)
            
            # Calculating gradients
            torch.autograd.backward([mass_loss, force_loss])
            
            # Update parameters
            optimizer.step()

            mass_total_loss += mass_loss.item()
            force_total_loss += force_loss.item()

        epoch_losses[0].append(mass_total_loss / len(train_loader))
        epoch_losses[1].append(force_total_loss / len(train_loader))

        model.eval()
        train_mass_accuracy, train_force_accuracy = evaluate(model, train_loader, seq_end=seq_end, step_size=step_size)
        epoch_accuracies[0].append((train_mass_accuracy, train_force_accuracy))
        val_mass_accuracy, val_force_accuracy = evaluate(model, val_loader, seq_end=seq_end, step_size=step_size)
        epoch_accuracies[1].append((val_mass_accuracy, val_force_accuracy))

        if val_mass_accuracy > best_mass_accuracy:
            best_mass_model = deepcopy(model)
            best_mass_accuracy = val_mass_accuracy
        
        if val_force_accuracy > best_force_accuracy:
            best_force_model = deepcopy(model)
            best_force_accuracy = val_force_accuracy
            
        if print_stats_per_epoch:
            pbar.set_description("Mass_loss (%.2f) Mass Train_acc (%.2f) Mass Val_acc (%.2f) Force_loss (%.2f)  Force Train_acc (%.2f) Force Val_acc (%.2f)" % 
                                 (epoch_losses[0][-1], train_mass_accuracy, val_mass_accuracy, 
                                  epoch_losses[1][-1], train_force_accuracy, val_force_accuracy))
    
    return epoch_losses, epoch_accuracies, [best_mass_model, best_force_model]

In [8]:
def get_current_prediction_and_number_of_correct_examples(y_hat, y_val):
    current_prediction = torch.max(y_hat.data, 1)[1]
    correct = (current_prediction == y_val).sum().cpu().numpy()
    return current_prediction, correct

def evaluate(model, val_loader, return_predicted=False, seq_start=None, seq_end=None, step_size=None):
    predicted = [[], []]
    accuracies = [[], []]

    mass_correct = 0
    force_correct = 0
    total = 0
    
    for x_val, y_val in val_loader:
        total += y_val.size(0)
        mass_val = y_val[:, 0]
        force_val = y_val[:, 1]

        x_val = Variable(x_val[:, seq_start:seq_end:step_size, :])
        mass_hat, force_hat = model(x_val)
        
        current_prediction, correct = get_current_prediction_and_number_of_correct_examples(mass_hat, mass_val)
        mass_correct += correct
        predicted[0].extend(current_prediction)
        
        current_prediction, correct = get_current_prediction_and_number_of_correct_examples(force_hat, force_val)
        force_correct += correct
        predicted[1].extend(current_prediction)
        
    accuracies[0] = 100 * mass_correct / float(total)
    accuracies[1] = 100 * force_correct / float(total)
    
    if return_predicted:
        return accuracies[0], accuracies[1], predicted[0], predicted[1]
    
    return accuracies[0], accuracies[1]

## Train model and plot loss and accuracy

In [9]:
def initialise_model(network_params, lr=0.01, seed=0, device=torch.device("cpu")):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    model = MultiBranchModel(*network_params)
    model = model.to(device=device)

    mass_error = nn.CrossEntropyLoss().to(device=device)
    force_error = nn.CrossEntropyLoss().to(device=device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    return model, mass_error, force_error, optimizer

In [10]:
def get_best_model_and_its_accuracy(accuracies, model, best_model, best_accuracy):
    model_accuracy = max(accuracies)
    print(model_accuracy)
    if model_accuracy > best_accuracy:
        return deepcopy(model), model_accuracy

    return best_model, best_accuracy

In [11]:
stats_dfs = []
loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                  class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                  multiclass=True,
                                  batch_size=BATCH_SIZE, normalise_data=NORMALISE_DATA,
                                  device=device)

best_mass_model_overall = None
best_mass_accuracy = 0.
best_force_model_overall = None
best_force_accuracy = 0.

for seed in [0, 42, 72]:
    df = pd.DataFrame()


    model, mass_error, force_error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device)
    epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                        mass_error, force_error,
                                                                                        loaders[0], loaders[1], 
                                                                                        EPOCHS, seq_end=SEQ_END,
                                                                                        step_size=STEP_SIZE)

    train_accuracies = np.array(epoch_accuracies[0])
    val_accuracies = np.array(epoch_accuracies[1])
    
    best_mass_model_overall, best_mass_accuracy = get_best_model_and_its_accuracy(val_accuracies[:, 0], 
                                                                                  best_mass_model, 
                                                                                  best_mass_model_overall,
                                                                                  best_mass_accuracy)

    best_force_model_overall, best_force_accuracy = get_best_model_and_its_accuracy(val_accuracies[:, 1], 
                                                                                    best_force_model,
                                                                                    best_force_model_overall,
                                                                                    best_force_accuracy)

    df["Epoch"] = np.arange(EPOCHS)
    df["Mass Loss"] = epoch_losses[0]
    df["Force Loss"] = epoch_losses[1]
    df["Mass Train Accuracy"] = train_accuracies[:, 0]
    df["Mass Val Accuracy"] = val_accuracies[:, 0]
    df["Force Train Accuracy"] = train_accuracies[:, 1]
    df["Force Val Accuracy"] = val_accuracies[:,1]
    df["seed"] = str(seed)
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf("multibranch_plots/stats.h5", key="stats")

100%|██████████| 10/10 [00:00<00:00, 673.37it/s]
Mass_loss (0.97) Mass Train_acc (60.00) Mass Val_acc (30.00) Force_loss (1.03)  Force Train_acc (80.00) Force Val_acc (30.00): 100%|██████████| 2/2 [00:00<00:00,  3.66it/s]
  0%|          | 0/2 [00:00<?, ?it/s]

30.0
30.0


Mass_loss (0.98) Mass Train_acc (70.00) Mass Val_acc (10.00) Force_loss (1.01)  Force Train_acc (60.00) Force Val_acc (50.00): 100%|██████████| 2/2 [00:00<00:00,  4.07it/s]
  0%|          | 0/2 [00:00<?, ?it/s]

10.0
50.0


Mass_loss (1.00) Mass Train_acc (50.00) Mass Val_acc (30.00) Force_loss (1.03)  Force Train_acc (60.00) Force Val_acc (40.00): 100%|██████████| 2/2 [00:00<00:00,  3.88it/s]

30.0
40.0


In [12]:
best_mass_accuracy, best_force_accuracy

(30.0, 50.0)

## Save model and scaler

In [13]:
torch.save(best_mass_model_overall.state_dict(), "models/passive_mass_dual_model.pt")
torch.save(best_force_model_overall.state_dict(), "models/passive_force_dual_model.pt")

In [14]:
joblib.dump(scaler, "scalers/passive_dual_scaler.sk")

['scalers/passive_dual_scaler.sk']